# NER mit spaCy

Eine allererste Zeile code:

In [ ]:
print("Hello World!")

### spacy importieren

In [ ]:
import spacy

Modell importieren (siehe in diesem Fall: https://spacy.io/models/en).

In [ ]:
nlp=spacy.load("en_core_web_sm")

Einen Text definieren

In [ ]:
text = "Berlin is a city in Germany and the State-Library Berlin is the national library of a state no longer in existence (Prussia)"


Spacy macht aus dem Text ein sog. Doc Objekt

In [ ]:
doc = nlp(text)

NER auf dem Doc Objekt:

In [ ]:
for ent in doc.ents:
    print (ent.text, ent.label_)

Im Hintergrund läuft dabei eine Spacy-Pipeline; NER ist nur ein (kleiner) Teil dieser "pipe".

In [ ]:
nlp.analyze_pipes()

Noch einmal auf Deutsch:

In [ ]:
text = "Berlin ist eine Stadt in Deutschland. Die Staatsbibliothek zu Berlin ist die Nationalbibliothek eines Staates, den es nicht mehr gibt (Preußen)"

In [ ]:
doc = nlp(text)

In [ ]:
for ent in doc.ents:
    print (ent.text, ent.label_)

In [ ]:
nlp = spacy.load("de_core_news_sm")

In [ ]:
doc = nlp(text)

In [ ]:
for ent in doc.ents:
    print (ent.text, ent.label_)

In [ ]:
#Eine regelbasierte Entity hinzufügen
ruler = nlp.add_pipe("entity_ruler", before="ner")

#Liste von Entities und Patterns
patterns = [
                {"label": "CITY", "pattern": "Berlin"},{"label": "ORG", "pattern": "Staatsbibliothek zu Berlin"}
            ]

ruler.add_patterns(patterns)

In [ ]:
doc = nlp(text)

# entities
for ent in doc.ents:
    print (ent.text, ent.label_)

### Visualisieren

In [ ]:
from spacy import displacy

In [ ]:
displacy.render(doc, style='ent', jupyter='true')

### Fonte, *Wanderungen*

In [ ]:
with open("data/fontane_brandenburg01_1862_ch1.txt") as f:
    fontane=f.read()

doc = nlp (fontane)

# Die ersten 30 entities in unserem ersten Kapitel
for ent in doc.ents[:30]:
    print (ent.text, ent.label_)

In [ ]:
# Der ganze Text, visualisiert:
displacy.render(doc, style='ent', jupyter='true')

Wir sehen: Das Modell müsste angepasst werden. Das größere Modell in spaCy würde ebenfalls deutlich bessere Ergebnisse liefern.

## Ein Modell mit spaCy trainieren

(Das Folgende basiert auf: W.J.B. Mattingly, Introduction to Named Entity Recognition, 2021 (2nd ed.).  https://ner.pythonhumanities.com/intro.html)

Trainingsdaten für spaCy NER müssen im folgenden Format stehen

TRAIN_DATA = [ (TEXT AS A STRING, {“entities”: [(START, END, LABEL)]}) ]

ACHTUNG: Startindizes beginnen mit "0", nicht mit "1"

In [ ]:
nlp = spacy.load("de_core_news_sm")
text = "Berlin ist eine Stadt in Deutschland. Die Staatsbibliothek zu Berlin ist die Nationalbibliothek eines Staates, den es nicht mehr gibt (Preußen)"


doc = nlp(text)

for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
corpus = []


for sent in doc.sents:
    corpus.append(sent.text)

nlp = spacy.blank("de")

ruler = nlp.add_pipe("entity_ruler")

patterns = [
                {"label": "CITY", "pattern": "Berlin"},
                {"label": "LOC", "pattern": "Deutschland"},
                {"label": "ORG", "pattern": "Staatsbibliothek zu Berlin"},
                {"label": "LOC", "pattern": "Preußen"},
            ]

ruler.add_patterns(patterns)

TRAIN_DATA = []
for sentence in corpus:
    doc = nlp(sentence)
    entities = []

    for ent in doc.ents:
        entities.append([ent.start_char, ent.end_char, ent.label_])
    TRAIN_DATA.append([sentence, {"entities": entities}])

print (TRAIN_DATA)

In [ ]:
import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [ ]:
convert("de", TRAIN_DATA, "data/train.spacy")
convert("de", TRAIN_DATA, "data/valid.spacy")

Konfigurationsdatei mit SpaCy erstellen: https://spacy.io/usage/training
Und die obigen Pfase für Trainings- und Validierungsset nachtragen.

Unser Beispiel ist BAD PRACTICE: Das Trainingsset ist natürlich viel zu klein und ist darüber hinaus identisch mit dem Validierungsset!

In [ ]:
!python -m spacy init fill-config data/base_config.cfg data/config.cfg

In [ ]:
!python -m spacy train data/config.cfg --output ./models/output

In [ ]:
trained_nlp = spacy.load("models/output/model-best")
text = "Berlin ist die Hauptstadt von Deutschland und die Staatsbibliothek von Berlin ist eine große wissenschaftliche Bibliothek."
doc = trained_nlp(text)

for ent in doc.ents:
    print (ent.text, ent.label_)